In [1]:
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
# import the pickle file that contains training, validating, and testing dataset
pickle_file = 'breast.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (17280, 40, 40), (17280,))
('Validation set', (2880, 40, 40), (2880,))
('Test set', (2880, 40, 40), (2880,))


In [3]:
# reformat the data to have the correct dimension
num_labels = 2
image_size = 40

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (17280, 1600), (17280, 2))
('Validation set', (2880, 1600), (2880, 2))
('Test set', (2880, 1600), (2880, 2))


In [4]:
# calculate the accuracy of prediction as compared to the true labels
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
# generate the confusion matrix, and calculate the prediction probability
# returns a list of tuples, each tuple has the format: (true label, model prediction, prediction probability)
# labels = 0 means benign, corresponds to (1, 0) in the output
# labels = 1 means malignant, corresponds to (0, 1) in the output
# threshold for classification is 0.5, meaning if the predicted value is bigger than 0.5, it's classified as malignant; if the predicted value is smaller than 0.5, it's classified as benign.  
def confusion(predictions, labels):   
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    prob_list = []
    prob = 0
    for i in range(len(labels)):
        if labels[i][1] > labels[i][0] and predictions[i][1] > predictions[i][0]: 
            TP += 1
            prob = (1, 1, predictions[i][1] / (predictions[i][1] + predictions[i][0]))
        elif labels[i][1] < labels[i][0] and predictions[i][1] < predictions[i][0]: 
            TN += 1
            prob = (0, 0, predictions[i][0] / (predictions[i][1] + predictions[i][0]))
        elif labels[i][1] < labels[i][0] and predictions[i][1] > predictions[i][0]: 
            FP += 1
            prob = (0, 1, predictions[i][1] / (predictions[i][1] + predictions[i][0]))
        elif labels[i][1] > labels[i][0] and predictions[i][1] < predictions[i][0]: 
            FN += 1
            prob = (1, 0, predictions[i][0] / (predictions[i][1] + predictions[i][0]))
        prob_list.append(prob)
    print('TP, TN, FP, FN: (%d, %d, %d, %d)' %(TP, TN, FP, FN))
    print('sensitivity: {:.1%}'.format(TP/float(TP+FN)))
    print('specificity: {:.1%}'.format(TN/float(TN+FP)))
    print('precision: {:.1%}'.format(TP/float(TP+FP)))
    print('F1 score: {:.1%}'.format(2*TP/float(TP+FP)*TP/float(TP+FN) / (TP/float(TP+FP)+TP/float(TP+FN))))
    return prob_list

In [6]:
# build the tensorflow graph
batch_size = 64
num_hidden_nodes1 = 512
num_hidden_nodes2 = 256
num_hidden_nodes3 = 128
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)

    # Variables.
    weights1 = tf.Variable(
    tf.truncated_normal(
        [image_size * image_size, num_hidden_nodes1],
        stddev=np.sqrt(2.0 / (image_size * image_size))))
    biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
    weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes1)))
    biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
    weights3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_hidden_nodes3], stddev=np.sqrt(2.0 / num_hidden_nodes2)))
    biases3 = tf.Variable(tf.zeros([num_hidden_nodes3]))
    weights4 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes3, num_labels], stddev=np.sqrt(2.0 / num_hidden_nodes3)))
    biases4 = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
    drop1 = tf.nn.dropout(lay1_train, keep_prob)
    lay2_train = tf.nn.relu(tf.matmul(drop1, weights2) + biases2)
    drop2 = tf.nn.dropout(lay2_train, keep_prob)
    lay3_train = tf.nn.relu(tf.matmul(drop2, weights3) + biases3)
    drop3 = tf.nn.dropout(lay3_train, keep_prob)
    logits = tf.matmul(drop3, weights4) + biases4
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    # Optimizer.
    learning_rate = tf.train.exponential_decay(0.1, global_step, 10000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
    lay2_valid = tf.nn.relu(tf.matmul(lay1_valid, weights2) + biases2)
    lay3_valid = tf.nn.relu(tf.matmul(lay2_valid, weights3) + biases3)
    valid_prediction = tf.nn.softmax(tf.matmul(lay3_valid, weights4) + biases4)
    lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
    lay2_test = tf.nn.relu(tf.matmul(lay1_test, weights2) + biases2)
    lay3_test = tf.nn.relu(tf.matmul(lay2_test, weights3) + biases3)
    test_prediction = tf.nn.softmax(tf.matmul(lay3_test, weights4) + biases4)

In [7]:
# first iteratively train the model
# then calculate the prediction accuracy on the test set using the trained model
# and generate the confusion matrix
num_steps = 20001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 1000 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))
    prob = confusion(test_prediction.eval(), test_labels)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 0.794202
Minibatch accuracy: 45.3%
Validation accuracy: 50.0%
Minibatch loss at step 1000: 0.589566
Minibatch accuracy: 68.8%
Validation accuracy: 67.2%
Minibatch loss at step 2000: 0.580320
Minibatch accuracy: 73.4%
Validation accuracy: 71.0%
Minibatch loss at step 3000: 0.518380
Minibatch accuracy: 79.7%
Validation accuracy: 71.8%
Minibatch loss at step 4000: 0.490529
Minibatch accuracy: 76.6%
Validation accuracy: 73.3%
Minibatch loss at step 5000: 0.459865
Minibatch accuracy: 76.6%
Validation accuracy: 79.2%
Minibatch loss at step 6000: 0.509324
Minibatch accuracy: 68.8%
Validation accuracy: 85.3%
Minibatch loss at step 7000: 0.356538
Minibatch accuracy: 79.7%
Validation accuracy: 87.5%
Minibatch loss at step 8000: 0.401502
Minibatch accuracy: 81.2%
Validation accuracy: 81.0%
Minibatch loss at step 9000: 0.316557
Minibatch accuracy: 84.4%
Validation accuracy: 89.1%
Minibat

In [8]:
# print the first 10 predictions with the corresponding probabilities.
# each tuple has the format: (true label, model prediction, prediction probability)
print(prob[:10])

[(0, 1, 0.78742182), (0, 1, 0.78058207), (0, 0, 0.99999988), (1, 0, 0.70625418), (1, 1, 0.99889141), (0, 0, 0.99458873), (1, 0, 0.98656958), (1, 1, 0.99986279), (0, 1, 0.76437426), (0, 1, 0.58809316)]
